<a href="https://colab.research.google.com/github/Ismat-Samadov/Named_Entity_Recognition/blob/main/notebooks/train_trocr_iam_correct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dataset Structure Understanding:
Each IAM image contains **TWO parts**:
1. **TOP half**: Machine-printed text (ground truth)
2. **BOTTOM half**: Handwritten copy of the same text

### Our Approach:
1. Split each image horizontally (top/bottom)
2. OCR the printed text (top) to extract ground truth labels
3. Use handwritten part (bottom) as training input
4. Train TrOCR: Handwritten Image → Printed Text

## 1. Setup and Installation

Install required packages for Google Colab

In [ ]:
# Install required packages
!pip install -q kagglehub transformers datasets pillow opencv-python-headless
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q pytesseract jiwer easyocr
!apt-get install -y tesseract-ocr

print("✓ All packages installed successfully!")

In [ ]:
# Imports
import sys
import os
from pathlib import Path
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from tqdm.auto import tqdm
from jiwer import cer, wer
import warnings
warnings.filterwarnings('ignore')

# Try importing EasyOCR for better printed text extraction
try:
    import easyocr
    USE_EASYOCR = True
except ImportError:
    USE_EASYOCR = False
    print("EasyOCR not available, will use pytesseract")

# Set random seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)

print("✓ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'}")

In [ ]:
# Analyze dataset statistics
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from collections import Counter

# Get all image files
all_images = list(data_dir.glob("**/*.png"))
if len(all_images) == 0:
    all_images = list(data_dir.glob("*.png"))

print("=" * 80)
print("IAM DATASET STATISTICS")
print("=" * 80)
print(f"Total images: {len(all_images)}")

# Analyze image dimensions
widths = []
heights = []
aspect_ratios = []

sample_size = min(100, len(all_images))
for img_path in all_images[:sample_size]:
    img = Image.open(img_path)
    w, h = img.size
    widths.append(w)
    heights.append(h)
    aspect_ratios.append(w/h)

print(f"\nImage Dimensions (sampled {sample_size} images):")
print(f"  Width:  {np.mean(widths):.0f} ± {np.std(widths):.0f} px (range: {min(widths)}-{max(widths)})")
print(f"  Height: {np.mean(heights):.0f} ± {np.std(heights):.0f} px (range: {min(heights)}-{max(heights)})")
print(f"  Aspect Ratio: {np.mean(aspect_ratios):.2f} ± {np.std(aspect_ratios):.2f}")

# Analyze directory structure
writers = {}
for img_path in all_images:
    if img_path.parent != data_dir:
        writer_id = img_path.parent.name
        if writer_id not in writers:
            writers[writer_id] = 0
        writers[writer_id] += 1

if writers:
    print(f"\nWriter Distribution:")
    print(f"  Total writers: {len(writers)}")
    print(f"  Images per writer: {np.mean(list(writers.values())):.1f} ± {np.std(list(writers.values())):.1f}")
    print(f"  Min images per writer: {min(writers.values())}")
    print(f"  Max images per writer: {max(writers.values())}")

# Visualize distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Width distribution
axes[0].hist(widths, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Width (pixels)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Image Width Distribution')
axes[0].grid(True, alpha=0.3)

# Height distribution
axes[1].hist(heights, bins=30, color='lightcoral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Height (pixels)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Image Height Distribution')
axes[1].grid(True, alpha=0.3)

# Aspect ratio distribution
axes[2].hist(aspect_ratios, bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
axes[2].set_xlabel('Aspect Ratio (W/H)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Aspect Ratio Distribution')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("DATASET CHARACTERISTICS:")
print("=" * 80)
print("✓ IAM Handwritten Forms Dataset")
print("✓ Contains handwritten copies of printed text")
print("✓ Each image has printed (top) and handwritten (bottom) versions")
print("✓ Multiple writers with different handwriting styles")
print("✓ Useful for training handwriting recognition models")
print("=" * 80)

In [ ]:
# Display sample images from the dataset
import random

# Get random sample of images
sample_images = random.sample(all_images, min(6, len(all_images)))

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, img_path in enumerate(sample_images):
    img = Image.open(img_path)
    axes[idx].imshow(img, cmap='gray')

    # Get writer ID if available
    if img_path.parent != data_dir:
        writer_info = f"Writer: {img_path.parent.name}"
    else:
        writer_info = "Dataset Sample"

    axes[idx].set_title(f"{writer_info}\nSize: {img.size[0]}x{img.size[1]}", fontsize=10)
    axes[idx].axis('off')

    # Add annotation for the split
    height = img.size[1]
    axes[idx].axhline(y=height//2, color='red', linestyle='--', linewidth=1.5, alpha=0.7)

plt.suptitle('Sample Images from IAM Dataset (Red line shows printed/handwritten split)',
             fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

print("\n✓ Sample images displayed successfully!")
print("Note: The red dashed line shows where we'll split the image:")
print("  - TOP half: Printed text (used as ground truth labels)")
print("  - BOTTOM half: Handwritten text (used as training input)")

## Sample Images from Dataset

Let's visualize some sample images to understand the dataset structure

## Dataset Analysis and Statistics

Let's analyze the IAM dataset to understand its characteristics

In [ ]:
# Imports
import sys
import os
from pathlib import Path
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from tqdm.auto import tqdm
from jiwer import cer, wer
import warnings
warnings.filterwarnings('ignore')

# Try importing EasyOCR for better printed text extraction
try:
    import easyocr
    USE_EASYOCR = True
except ImportError:
    USE_EASYOCR = False
    print("EasyOCR not available, will use pytesseract")

# Set random seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 8)

print("✓ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'}")

ImportError: dlopen(/Users/ismatsamadov/handwriting_data_processing/venv/lib/python3.10/site-packages/torch/_C.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/libtorch_cpu.dylib
  Referenced from: <8C033F9E-F29C-3CF8-80FD-7D03760F3A30> /Users/ismatsamadov/handwriting_data_processing/venv/lib/python3.10/site-packages/torch/lib/libtorch_python.dylib
  Reason: tried: '/Users/ismatsamadov/handwriting_data_processing/venv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib' (no such file), '/Users/runner/work/_temp/anaconda/envs/wheel_py310/lib/libtorch_cpu.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/work/_temp/anaconda/envs/wheel_py310/lib/libtorch_cpu.dylib' (no such file), '/Users/ismatsamadov/handwriting_data_processing/venv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib' (no such file), '/Users/runner/work/_temp/anaconda/envs/wheel_py310/lib/libtorch_cpu.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/w

# Load and display sample images to understand structure
# Use data_dir from the download step
sample_images = list(data_dir.glob("*/*.png"))[:6]

if len(sample_images) == 0:
    # Try without subdirectories
    sample_images = list(data_dir.glob("*.png"))[:6]

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, img_path in enumerate(sample_images):
    img = Image.open(img_path)
    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(f"Writer {img_path.parent.name} - {img_path.name}")
    axes[idx].axis('off')
    
    # Draw a horizontal line to show the split point
    height = img.size[1]
    axes[idx].axhline(y=height//2, color='red', linestyle='--', linewidth=2, label='Split line')
    axes[idx].text(10, height//4, 'PRINTED', color='blue', fontsize=12, weight='bold',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    axes[idx].text(10, 3*height//4, 'HANDWRITTEN', color='green', fontsize=12, weight='bold',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('IAM Dataset Structure: TOP = Printed, BOTTOM = Handwritten', fontsize=16, weight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("DATASET STRUCTURE CONFIRMED:")
print("="*80)
print("✓ Each image has TWO parts:")
print("  1. TOP half    : Machine-printed text (GROUND TRUTH)")
print("  2. BOTTOM half : Handwritten text (INPUT for training)")
print("="*80)

In [ ]:
# Load and display sample images to understand structure
data_dir = Path("../archive/data")
sample_images = list(data_dir.glob("*/*.png"))[:6]

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, img_path in enumerate(sample_images):
    img = Image.open(img_path)
    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(f"Writer {img_path.parent.name} - {img_path.name}")
    axes[idx].axis('off')

    # Draw a horizontal line to show the split point
    height = img.size[1]
    axes[idx].axhline(y=height//2, color='red', linestyle='--', linewidth=2, label='Split line')
    axes[idx].text(10, height//4, 'PRINTED', color='blue', fontsize=12, weight='bold',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    axes[idx].text(10, 3*height//4, 'HANDWRITTEN', color='green', fontsize=12, weight='bold',
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('IAM Dataset Structure: TOP = Printed, BOTTOM = Handwritten', fontsize=16, weight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("DATASET STRUCTURE CONFIRMED:")
print("="*80)
print("✓ Each image has TWO parts:")
print("  1. TOP half    : Machine-printed text (GROUND TRUTH)")
print("  2. BOTTOM half : Handwritten text (INPUT for training)")
print("="*80)

## 3. Image Splitting Functions

Create functions to split images and extract text from printed part

In [ ]:
def split_image(image_path):
    """
    Split IAM image into printed (top) and handwritten (bottom) parts.

    Args:
        image_path: Path to IAM image

    Returns:
        Tuple of (printed_image, handwritten_image)
    """
    img = Image.open(image_path).convert('RGB')
    width, height = img.size

    # Split at middle
    split_point = height // 2

    # Top half = printed text
    printed_img = img.crop((0, 0, width, split_point))

    # Bottom half = handwritten text
    handwritten_img = img.crop((0, split_point, width, height))

    return printed_img, handwritten_img


def extract_text_from_printed(printed_img, method='easyocr'):
    """
    Extract text from printed portion using OCR.

    Args:
        printed_img: PIL Image of printed text
        method: 'easyocr' or 'tesseract'

    Returns:
        Extracted text string
    """
    if method == 'easyocr' and USE_EASYOCR:
        # Use EasyOCR (more accurate for printed text)
        reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        img_np = np.array(printed_img)
        results = reader.readtext(img_np, detail=0)
        text = ' '.join(results)
    else:
        # Fallback to Tesseract
        import pytesseract
        text = pytesseract.image_to_string(printed_img)
        text = text.strip()

    # Clean up text
    text = ' '.join(text.split())  # Remove extra whitespace
    return text


# Test the functions
print("Testing image splitting...\n")
test_img_path = sample_images[0]
printed, handwritten = split_image(test_img_path)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Original
axes[0].imshow(Image.open(test_img_path), cmap='gray')
axes[0].set_title('Original Image', fontsize=14, weight='bold')
axes[0].axis('off')

# Printed
axes[1].imshow(printed, cmap='gray')
axes[1].set_title('Printed Text (TOP) → Ground Truth', fontsize=14, weight='bold', color='blue')
axes[1].axis('off')

# Handwritten
axes[2].imshow(handwritten, cmap='gray')
axes[2].set_title('Handwritten Text (BOTTOM) → Training Input', fontsize=14, weight='bold', color='green')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n✓ Image splitting successful!")
print(f"Printed image size: {printed.size}")
print(f"Handwritten image size: {handwritten.size}")

## 4. Extract Ground Truth Labels

Use OCR to extract text from printed portions

In [ ]:
# Initialize EasyOCR reader (do this once)
if USE_EASYOCR:
    print("Initializing EasyOCR...")
    ocr_reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
    print("✓ EasyOCR ready!")
else:
    print("Using Tesseract OCR")
    ocr_reader = None

# Test OCR extraction
print("\nTesting text extraction from printed portion...\n")
test_printed, test_handwritten = split_image(sample_images[0])

# Extract text
if USE_EASYOCR:
    img_np = np.array(test_printed)
    results = ocr_reader.readtext(img_np, detail=0)
    ground_truth = ' '.join(results)
else:
    import pytesseract
    ground_truth = pytesseract.image_to_string(test_printed).strip()

ground_truth = ' '.join(ground_truth.split())  # Clean

print("="*80)
print("EXTRACTED GROUND TRUTH:")
print("="*80)
print(f"{ground_truth}")
print("="*80)
print(f"\nLength: {len(ground_truth)} characters")

## 5. Create Correct IAM Dataset Class

In [ ]:
class IAMHandwritingDataset(Dataset):
    """
    IAM Dataset with correct understanding:
    - Splits each image into printed (top) and handwritten (bottom)
    - Extracts ground truth from printed text
    - Uses handwritten part for training
    """

    def __init__(self, data_dir, split='train', processor=None, num_samples=None, cache_labels=True):
        """
        Args:
            data_dir: Path to archive/data
            split: 'train', 'val', or 'test'
            processor: TrOCR processor
            num_samples: Limit samples (for testing)
            cache_labels: Cache extracted labels to speed up training
        """
        self.data_dir = Path(data_dir)
        self.split = split
        self.processor = processor
        self.cache_labels = cache_labels
        self.label_cache = {}

        # Initialize OCR reader
        if USE_EASYOCR:
            self.ocr_reader = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        else:
            self.ocr_reader = None

        # Load image paths
        all_images = list(self.data_dir.glob("*/*.png"))
        random.seed(42)
        random.shuffle(all_images)

        # Split into train/val/test
        total = len(all_images)
        train_end = int(total * 0.8)
        val_end = int(total * 0.9)

        if split == 'train':
            selected = all_images[:train_end]
        elif split == 'val':
            selected = all_images[train_end:val_end]
        else:  # test
            selected = all_images[val_end:]

        if num_samples:
            selected = selected[:num_samples]

        self.image_paths = selected
        print(f"✓ Loaded {len(self.image_paths)} images for {split} split")

    def _extract_label(self, image_path):
        """Extract ground truth label from printed portion."""
        # Check cache
        if self.cache_labels and str(image_path) in self.label_cache:
            return self.label_cache[str(image_path)]

        # Split image
        printed_img, _ = split_image(image_path)

        # Extract text using OCR
        if USE_EASYOCR and self.ocr_reader:
            img_np = np.array(printed_img)
            results = self.ocr_reader.readtext(img_np, detail=0)
            text = ' '.join(results)
        else:
            import pytesseract
            text = pytesseract.image_to_string(printed_img).strip()

        # Clean text
        text = ' '.join(text.split())

        # Cache
        if self.cache_labels:
            self.label_cache[str(image_path)] = text

        return text

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        # Split image and get handwritten part
        _, handwritten_img = split_image(image_path)

        # Extract ground truth from printed part
        text = self._extract_label(image_path)

        # If text is empty or too short, skip
        if len(text.strip()) < 3:
            text = "sample text"  # Fallback

        if self.processor:
            # Process handwritten image
            pixel_values = self.processor(handwritten_img, return_tensors="pt").pixel_values.squeeze()

            # Process text label
            labels = self.processor.tokenizer(
                text,
                padding="max_length",
                max_length=128,
                truncation=True,
                return_tensors="pt"
            ).input_ids.squeeze()

            # Replace padding with -100 for loss calculation
            labels[labels == self.processor.tokenizer.pad_token_id] = -100

            return {
                'pixel_values': pixel_values,
                'labels': labels,
                'text': text
            }
        else:
            return {
                'image': handwritten_img,
                'text': text
            }


print("✓ Dataset class created!")

## 6. Initialize TrOCR Model

In [ ]:
# Load TrOCR model and processor
print("Loading TrOCR model...")
model_name = "microsoft/trocr-base-handwritten"
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# IMPORTANT: Set decoder_start_token_id and pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

model.to(device)

print(f"✓ Model loaded on {device}")
print(f"\nModel Architecture:")
print(f"  Encoder: {model.config.encoder.model_type} ({model.encoder.num_parameters():,} params)")
print(f"  Decoder: {model.config.decoder.model_type} ({model.decoder.num_parameters():,} params)")
print(f"  Total Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Decoder Start Token ID: {model.config.decoder_start_token_id}")
print(f"  Pad Token ID: {model.config.pad_token_id}")

# Create datasets (start with small sample for testing)
print("Creating datasets...\n")

train_dataset = IAMHandwritingDataset(
    data_dir,  # Use the downloaded dataset path
    split='train',
    processor=processor,
    num_samples=50  # Start small for testing
)

val_dataset = IAMHandwritingDataset(
    data_dir,  # Use the downloaded dataset path
    split='val',
    processor=processor,
    num_samples=10
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)

print(f"\n✓ DataLoaders created")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Val samples: {len(val_dataset)}")
print(f"  Batch size: 4")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")

# Show sample batch
sample_batch = next(iter(train_loader))
print(f"\nSample Batch:")
print(f"  pixel_values shape: {sample_batch['pixel_values'].shape}")
print(f"  labels shape: {sample_batch['labels'].shape}")
print(f"\nSample Ground Truth Texts:")
for i, text in enumerate(sample_batch['text'][:3]):
    print(f"  {i+1}. {text}")

In [ ]:
# Create datasets (start with small sample for testing)
print("Creating datasets...\n")

train_dataset = IAMHandwritingDataset(
    "../archive/data",
    split='train',
    processor=processor,
    num_samples=50  # Start small for testing
)

val_dataset = IAMHandwritingDataset(
    "../archive/data",
    split='val',
    processor=processor,
    num_samples=10
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)

print(f"\n✓ DataLoaders created")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Val samples: {len(val_dataset)}")
print(f"  Batch size: 4")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")

# Show sample batch
sample_batch = next(iter(train_loader))
print(f"\nSample Batch:")
print(f"  pixel_values shape: {sample_batch['pixel_values'].shape}")
print(f"  labels shape: {sample_batch['labels'].shape}")
print(f"\nSample Ground Truth Texts:")
for i, text in enumerate(sample_batch['text'][:3]):
    print(f"  {i+1}. {text}")

# Visualize training samples
fig, axes = plt.subplots(3, 2, figsize=(18, 12))
axes = axes.flatten()

# Get samples without processing
vis_dataset = IAMHandwritingDataset(
    data_dir,  # Use the downloaded dataset path
    split='train',
    processor=None,
    num_samples=6
)

for idx in range(6):
    sample = vis_dataset[idx]
    img = sample['image']
    text = sample['text']
    
    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(f"Ground Truth: {text[:50]}...", fontsize=10, wrap=True)
    axes[idx].axis('off')

plt.suptitle('Handwritten Images (Training Input) with Ground Truth Labels (from printed text)',
             fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("TRAINING SETUP:")
print("="*80)
print("INPUT:  Handwritten images (bottom half)")
print("OUTPUT: Text labels (extracted from printed top half)")
print("GOAL:   Train TrOCR to recognize handwriting")
print("="*80)

In [ ]:
# Visualize training samples
fig, axes = plt.subplots(3, 2, figsize=(18, 12))
axes = axes.flatten()

# Get samples without processing
vis_dataset = IAMHandwritingDataset(
    "../archive/data",
    split='train',
    processor=None,
    num_samples=6
)

for idx in range(6):
    sample = vis_dataset[idx]
    img = sample['image']
    text = sample['text']

    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(f"Ground Truth: {text[:50]}...", fontsize=10, wrap=True)
    axes[idx].axis('off')

plt.suptitle('Handwritten Images (Training Input) with Ground Truth Labels (from printed text)',
             fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("TRAINING SETUP:")
print("="*80)
print("INPUT:  Handwritten images (bottom half)")
print("OUTPUT: Text labels (extracted from printed top half)")
print("GOAL:   Train TrOCR to recognize handwriting")
print("="*80)

## 9. Training Setup

In [ ]:
# Training configuration
NUM_EPOCHS = 5
LEARNING_RATE = 5e-5
WARMUP_STEPS = 20

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=total_steps
)

print("Training Configuration:")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Warmup Steps: {WARMUP_STEPS}")
print(f"  Total Steps: {total_steps}")
print(f"  Optimizer: AdamW")
print(f"  Device: {device}")
print(f"\n  Dataset Understanding:")
print(f"    ✓ Handwritten images as input")
print(f"    ✓ Printed text (OCR extracted) as labels")
print(f"    ✓ Each sample: Handwritten → Printed Text")

## 10. Training Loop

In [ ]:
# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'val_cer': [],
    'val_wer': [],
    'learning_rate': []
}

print("="*70)
print("STARTING TRAINING")
print("="*70)

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch}/{NUM_EPOCHS}")
    print("="*70)

    # Training
    model.train()
    train_loss = 0
    progress_bar = tqdm(train_loader, desc="Training")

    for batch in progress_bar:
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

    avg_train_loss = train_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)
    history['learning_rate'].append(scheduler.get_last_lr()[0])

    # Validation
    model.eval()
    val_loss = 0
    all_preds = []
    all_refs = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(pixel_values=pixel_values, labels=labels)
            val_loss += outputs.loss.item()

            # Generate predictions
            generated_ids = model.generate(pixel_values, max_length=128)
            preds = processor.batch_decode(generated_ids, skip_special_tokens=True)
            refs = batch['text']

            all_preds.extend(preds)
            all_refs.extend(refs)

    avg_val_loss = val_loss / len(val_loader)

    # Calculate metrics
    try:
        val_cer = cer(all_refs, all_preds)
        val_wer = wer(all_refs, all_preds)
    except:
        val_cer = 0.0
        val_wer = 0.0

    history['val_loss'].append(avg_val_loss)
    history['val_cer'].append(val_cer)
    history['val_wer'].append(val_wer)

    # Print results
    print(f"\nEpoch {epoch} Results:")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val Loss:   {avg_val_loss:.4f}")
    print(f"  Val CER:    {val_cer:.4f} ({val_cer*100:.2f}%)")
    print(f"  Val WER:    {val_wer:.4f} ({val_wer*100:.2f}%)")

    # Show sample predictions
    print(f"\nSample Predictions:")
    for i in range(min(3, len(all_preds))):
        print(f"\n  [{i+1}]")
        print(f"  Reference:  {all_refs[i]}")
        print(f"  Prediction: {all_preds[i]}")

print("\n" + "="*70)
print("TRAINING COMPLETED!")
print("="*70)

# Plot training history
if history['train_loss']:
    # Create experiments directory in current working directory
    Path('./experiments').mkdir(parents=True, exist_ok=True)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    epochs_range = range(1, len(history['train_loss']) + 1)

    # Loss
    axes[0, 0].plot(epochs_range, history['train_loss'], 'o-', label='Train Loss', linewidth=2, markersize=8)
    axes[0, 0].plot(epochs_range, history['val_loss'], 's-', label='Val Loss', linewidth=2, markersize=8)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # CER
    axes[0, 1].plot(epochs_range, [c*100 for c in history['val_cer']], 'o-',
                    color='#e74c3c', linewidth=2, markersize=8)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('CER (%)')
    axes[0, 1].set_title('Character Error Rate')
    axes[0, 1].grid(True, alpha=0.3)

    # WER
    axes[1, 0].plot(epochs_range, [w*100 for w in history['val_wer']], 'o-',
                    color='#f39c12', linewidth=2, markersize=8)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('WER (%)')
    axes[1, 0].set_title('Word Error Rate')
    axes[1, 0].grid(True, alpha=0.3)

    # Learning Rate
    axes[1, 1].plot(range(len(history['learning_rate'])), history['learning_rate'],
                    color='#9b59b6', linewidth=2)
    axes[1, 1].set_xlabel('Step')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_title('Learning Rate Schedule')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('./experiments/training_metrics_correct.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✓ Plots saved to ./experiments/training_metrics_correct.png")
else:
    print("⚠ No training history available. Run training first.")

In [ ]:
# Plot training history
if history['train_loss']:
    # Create experiments directory
    Path('../experiments').mkdir(parents=True, exist_ok=True)

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    epochs_range = range(1, len(history['train_loss']) + 1)

    # Loss
    axes[0, 0].plot(epochs_range, history['train_loss'], 'o-', label='Train Loss', linewidth=2, markersize=8)
    axes[0, 0].plot(epochs_range, history['val_loss'], 's-', label='Val Loss', linewidth=2, markersize=8)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # CER
    axes[0, 1].plot(epochs_range, [c*100 for c in history['val_cer']], 'o-',
                    color='#e74c3c', linewidth=2, markersize=8)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('CER (%)')
    axes[0, 1].set_title('Character Error Rate')
    axes[0, 1].grid(True, alpha=0.3)

    # WER
    axes[1, 0].plot(epochs_range, [w*100 for w in history['val_wer']], 'o-',
                    color='#f39c12', linewidth=2, markersize=8)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('WER (%)')
    axes[1, 0].set_title('Word Error Rate')
    axes[1, 0].grid(True, alpha=0.3)

    # Learning Rate
    axes[1, 1].plot(range(len(history['learning_rate'])), history['learning_rate'],
                    color='#9b59b6', linewidth=2)
    axes[1, 1].set_xlabel('Step')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_title('Learning Rate Schedule')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('../experiments/training_metrics_correct.png', dpi=300, bbox_inches='tight')
    plt.show()

    print("✓ Plots saved to experiments/training_metrics_correct.png")
else:
    print("⚠ No training history available. Run training first.")

# Save model
output_dir = "./experiments/trocr_iam_correct"
Path(output_dir).mkdir(parents=True, exist_ok=True)

model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

print(f"✓ Model saved to {output_dir}")

# Save training history
import json
with open(f"{output_dir}/training_history.json", 'w') as f:
    json.dump(history, f, indent=2)

print(f"✓ Training history saved to {output_dir}/training_history.json")

# Optional: Mount Google Drive to save permanently
print("\n" + "="*80)
print("TIP: To save to Google Drive, run:")
print("  from google.colab import drive")
print("  drive.mount('/content/drive')")
print("  Then copy the experiments folder to your Drive")
print("="*80)

In [ ]:
# Save model
output_dir = "../experiments/trocr_iam_correct"
Path(output_dir).mkdir(parents=True, exist_ok=True)

model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

print(f"✓ Model saved to {output_dir}")

# Save training history
import json
with open(f"{output_dir}/training_history.json", 'w') as f:
    json.dump(history, f, indent=2)

print(f"✓ Training history saved to {output_dir}/training_history.json")

# Test on a sample
model.eval()

# Get a test sample
test_dataset = IAMHandwritingDataset(
    data_dir,  # Use the downloaded dataset path
    split='test',
    processor=None,
    num_samples=3
)

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for idx in range(3):
    sample = test_dataset[idx]
    handwritten_img = sample['image']
    ground_truth = sample['text']
    
    # Predict
    pixel_values = processor(handwritten_img, return_tensors="pt").pixel_values.to(device)
    
    with torch.no_grad():
        generated_ids = model.generate(pixel_values, max_length=128, num_beams=5)
        predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # Display
    axes[idx].imshow(handwritten_img, cmap='gray')
    axes[idx].set_title(
        f"Ground Truth: {ground_truth}\nPredicted: {predicted_text}",
        fontsize=11, loc='left'
    )
    axes[idx].axis('off')

plt.suptitle('Model Predictions on Test Set', fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("INFERENCE TEST COMPLETE")
print("="*80)

In [ ]:
# Test on a sample
model.eval()

# Get a test sample
test_dataset = IAMHandwritingDataset(
    "../archive/data",
    split='test',
    processor=None,
    num_samples=3
)

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for idx in range(3):
    sample = test_dataset[idx]
    handwritten_img = sample['image']
    ground_truth = sample['text']

    # Predict
    pixel_values = processor(handwritten_img, return_tensors="pt").pixel_values.to(device)

    with torch.no_grad():
        generated_ids = model.generate(pixel_values, max_length=128, num_beams=5)
        predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Display
    axes[idx].imshow(handwritten_img, cmap='gray')
    axes[idx].set_title(
        f"Ground Truth: {ground_truth}\nPredicted: {predicted_text}",
        fontsize=11, loc='left'
    )
    axes[idx].axis('off')

plt.suptitle('Model Predictions on Test Set', fontsize=14, weight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("INFERENCE TEST COMPLETE")
print("="*80)

## 14. Summary

### What We Did Correctly:

1. ✅ **Understood IAM dataset structure**:
   - Top half = Printed text (ground truth)
   - Bottom half = Handwritten text (training input)

2. ✅ **Created proper data pipeline**:
   - Split images horizontally
   - Extract labels from printed text using OCR
   - Use handwritten images for training

3. ✅ **Trained TrOCR correctly**:
   - Input: Handwritten images
   - Output: Text transcriptions
   - Labels: Extracted from printed text

4. ✅ **Fixed configuration issues**:
   - Set decoder_start_token_id
   - Set pad_token_id

### Next Steps:

- Train on full dataset (not just 50 samples)
- Increase epochs (10-20 for better convergence)
- Fine-tune hyperparameters
- Integrate with ensemble pipeline
- Deploy to demo interface